# Data Cleaning Pornhub

*In this document you will find the process of data cleaning of Pornhub database.*

1st we import pandas and numpy and read the file.

In [24]:
import pandas as pd
import numpy as np

In [25]:
ph = pd.read_csv('/Users/Maria/Desktop/tumbzilla_labels.csv', index_col = 0)
ph.columns

Index(['img_source', 'length', 'nb_views', 'quality', 'title', 'video_link',
       'voting', 'categories', 'tags'],
      dtype='object')

We drop some columns since we will not be using throughout this project.

In [26]:
ph.drop(['img_source', 'video_link','tags', 'length', 'quality' ], axis = 1, inplace=True)

We check the column names and the data types, to see if we have to do some cleaning, we did not have to change

In [27]:
ph.columns

Index(['nb_views', 'title', 'voting', 'categories'], dtype='object')

In [28]:
ph.dtypes

nb_views      float64
title          object
voting        float64
categories     object
dtype: object

We defined a function to turn the string of categories into a list of categories in order do the analysis.

In [7]:
def cast_to_list(categories):
    if pd.isnull(categories):
        return categories
    return [item.strip() for item in categories.split("__")]

ph['categories']= ph['categories'].apply(cast_to_list)

In [8]:
ph.head()

,nb_views,title,voting,categories
0,127450.0,Tease then suck,84.0,[Blowjob]
1,480620.0,Two hot chick on one dick,79.0,[Threesome]
2,99720.0,Chick stretches her pussy and blows for the cam,77.0,[Brunette]
3,598820.0,Fucking my neighbour,80.0,"[Blonde, MILF]"
4,155850.0,Small titted whore sucking a cock for lunch,79.0,[POV]


In [9]:
categories = ph['categories'].apply(pd.Series) #We are saving the result into a new variable called categories. 

We drop some columns and stick our analysis to the 3 first categories to simplify our analysis and rename the columns.

In [10]:
categories.drop(columns = [3,4,5,6,7,8,9,10,11], inplace = True) #Dropping the columns

In [11]:
categories.rename(columns = {0:'Category_1', 1:'Category_2', 2:'Category_3'}, inplace = True) #Renaming the columns

We merge the resulting data with the original framework

In [ ]:
ph_merged = ph.merge(right = categories, left_index = True, right_index=True)

In [13]:
ph_merged['Website'] = "Pornhub" #Just checking that it worked
ph_merged.head(2)

,nb_views,title,voting,categories,Category_1,Category_2,Category_3,Website
0,127450.0,Tease then suck,84.0,[Blowjob],Blowjob,NaN,NaN,Pornhub
1,480620.0,Two hot chick on one dick,79.0,[Threesome],Threesome,NaN,NaN,Pornhub


We check that the Data Types are correct. We want nb_views to be integer so we apply astype method.

In [ ]:
ph_merged.dtypes #Checking the types

In [14]:
ph_merged.astype({'nb_views': 'int32'}).dtypes #changing data types

nb_views        int32
title          object
voting        float64
categories     object
Category_1     object
Category_2     object
Category_3     object
Website        object
dtype: object

Now we check the columns, drop categories column since we will not be working with it and we rename some columns to analyze better our data.

In [15]:
ph_merged.columns #Checking the columns

Index(['nb_views', 'title', 'voting', 'categories', 'Category_1', 'Category_2',
       'Category_3', 'Website'],
      dtype='object')

In [16]:
ph_merged.drop(columns = 'categories') #We drop categories column since we will be working with Category_1, Category_2, Category_3.

,nb_views,title,voting,Category_1,Category_2,Category_3,Website
0,127450.0,Tease then suck,84.0,Blowjob,NaN,NaN,Pornhub
1,480620.0,Two hot chick on one dick,79.0,Threesome,NaN,NaN,Pornhub
2,99720.0,Chick stretches her pussy and blows for the cam,77.0,Brunette,NaN,NaN,Pornhub
3,598820.0,Fucking my neighbour,80.0,Blonde,MILF,NaN,Pornhub
4,155850.0,Small titted whore sucking a cock for lunch,79.0,POV,NaN,NaN,Pornhub
...,...,...,...,...,...,...,...
191527,257540.0,Czech taxi 39,75.0,Public,Double Penetration,Czech,Pornhub
191528,73960.0,Casanova Holmes - Quality Vintage 1970s XXX,70.0,For Women,Vintage,Big Dick,Pornhub
191529,419960.0,19 Year Old Blonde Fuck in a threesome,74.0,Babe,Exclusive,Verified Amateurs,Pornhub
191530,216150.0,Virtual Sex - Anya Ivy,72.0,Big Tits,POV,Fetish,Pornhub


In [18]:
ph_merged = ph_merged[['Title', 'Number_of_Views', 'Rating', 'Category_1', 'Category_2', 'Category_3', 'Website']]

In [19]:
ph_merged.head(3)

,Title,Number_of_Views,Rating,Category_1,Category_2,Category_3,Website
0,Tease then suck,127450.0,84.0,Blowjob,NaN,NaN,Pornhub
1,Two hot chick on one dick,480620.0,79.0,Threesome,NaN,NaN,Pornhub
2,Chick stretches her pussy and blows for the cam,99720.0,77.0,Brunette,NaN,NaN,Pornhub


We export the information to the Database (MySql)

In [21]:
ph_merged.to_sql(name = 'main', con = connection_string)